In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, AdamW, get_scheduler, AutoModel

from pyswarms.single.global_best import GlobalBestPSO

In [2]:
# Load the CSV file into a DataFrame
df = pd.read_csv('Datasets/data_amazon_product_reviews_video_games.csv')

df.drop(labels= ['Unnamed: 0', 'reviewerID', 'asin', 'reviewerName', 'helpful',
       'unixReviewTime', 'reviewTime'], axis= 1, inplace= True)

df.dropna(inplace= True)

df['overall']= df['overall'].astype(dtype= 'int64')

df['new_text']= df['reviewText'] + ' ' + df['summary']

# Limitation of size of data
df = df.sample(n=1000, random_state=42)

texts= df['new_text'].tolist()
labels= df['overall'].tolist()

In [4]:
# Split the data into train and test sets (80% train, 20% test)
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size= 0.2, random_state= 42)

# Further split the test set into dev and test sets (50% dev, 50% test)
dev_texts, test_texts, dev_labels, test_labels = train_test_split(test_texts, test_labels, test_size= 0.5, random_state=42)

# Load the pre-trained BERT tokenizer and model
checkpoint= "LiYuan/amazon-review-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Tokenize the input texts
tokenized_train_texts = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')
tokenized_dev_texts = tokenizer(dev_texts, padding=True, truncation=True, return_tensors='pt')
tokenized_test_texts = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')

# Convert the labels to tensor
train_labels = torch.tensor(train_labels)
dev_labels = torch.tensor(dev_labels)
test_labels = torch.tensor(test_labels)

# Create TensorDatasets and DataLoaders for train, dev, and test sets
train_dataset = TensorDataset(tokenized_train_texts['input_ids'], tokenized_train_texts['attention_mask'], train_labels)
dev_dataset = TensorDataset(tokenized_dev_texts['input_ids'], tokenized_dev_texts['attention_mask'], dev_labels)
test_dataset = TensorDataset(tokenized_test_texts['input_ids'], tokenized_test_texts['attention_mask'], test_labels)


In [5]:
# Define your model architecture
class SentimentClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, output_dim, dropout_rate):
        super(SentimentClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(bert_model.config.hidden_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim= 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids= input_ids, attention_mask= attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [6]:
# Define the objective function
def objective_function(params):
    
    # Define hyperparameters
    dropout_rate = 0.1   # Dropout ratio of the additional hidden layer
    hidden_dim = 256   # 256 classes as an additional hidden layer
    output_dim = 5    # 5 classes for sentiment analysis

    Costs = np.zeros(params.shape[0])  # x.shape[0] gives the number of particles
    for i in range(params.shape[0]):
        learning_rate= params[i, 0]
        batch_size= int(params[i, 1])
        weight_decay= params[i, 2]
        num_epochs= int(params[i, 3])

    train_dataloader = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)
    dev_dataloader = DataLoader(dev_dataset, batch_size= batch_size, shuffle= False)
    test_dataloader = DataLoader(test_dataset, batch_size= batch_size, shuffle= False)
        
    # LiYuan Model
    bert_model = AutoModel.from_pretrained(checkpoint)

    # Freeze the BERT model parameters
    for param in bert_model.parameters():
        param.requires_grad = False
        
    # Create an instance of the SentimentClassifier
    model = SentimentClassifier(bert_model, hidden_dim, output_dim, dropout_rate)
        
    # Define the optimizer for training the softmax layer
    optimizer = optim.Adam(model.parameters(), lr= learning_rate, weight_decay= weight_decay)
        
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    # Define the loss function
    criterion = nn.CrossEntropyLoss()
        
    # Train the model
    for epoch in range(num_epochs):
        # Training loop
        model.train()
        for batch in train_dataloader:
            input_ids, attention_mask, batch_labels = batch
                
            input_ids= input_ids.to(device)
            attention_mask= attention_mask.to(device)
            batch_labels= batch_labels.to(device)
                
            optimizer.zero_grad()
            outputs = model(input_ids= input_ids, attention_mask= attention_mask)
            loss = criterion(outputs, batch_labels)  
            loss.backward()
            optimizer.step()
                
        # Validation loop
        model.eval()
        dev_correct = 0
        total_dev = 0
        y_true = []
        y_pred = []
        loss_epoch= []
        with torch.no_grad():
            for batch in dev_dataloader:
                input_ids, attention_mask, batch_labels = batch
                    
                input_ids= input_ids.to(device)
                attention_mask= attention_mask.to(device)
                batch_labels= batch_labels.to(device)
                    
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs
                _, predicted = torch.max(logits, 1)

                # Append true labels and predicted labels for later use
                y_true.extend(batch_labels.tolist())
                y_pred.extend(predicted.tolist())

                # Calculate the loss
                loss = criterion(logits, batch_labels)
                loss_epoch.append(loss)
                    
            
        # Calculate accuracy and F1 score
        f1 = f1_score(y_true, y_pred, average='weighted')
        accuracy = accuracy_score(y_true, y_pred)

        # Calculate the average loss
        loss_epoch_np = [tensor.cpu().detach().numpy() for tensor in loss_epoch]
        average_loss= np.mean(loss_epoch_np)
            
        print(f'learning_rate: {learning_rate}, batch_size: {batch_size}, weight_decay: {weight_decay}, num_epochs: {num_epochs}')
        print(f'epoch No. : {epoch}, Devset Accuracy : {round(accuracy,5)}, Devset f1_score : {round(f1,5)}, Average loss: {round(average_loss.tolist(),5)}')
        print()
    validation_loss= average_loss

    Costs[i]= (1/f1)**2
    
    # Return the validation loss as the objective value
    return Costs

In [7]:
# Define hyperparameters #learning_rate = 1e-3 #batch_size = 16 #weight_decay = 1e-4 #num_epochs = 1

# Define the bounds for the hyperparameters
lower_bound = np.array([1e-4, 16, 1e-5, 1])
upper_bound = np.array([1e-2, 256, 1e-3, 2])
bounds = (lower_bound, upper_bound)

In [8]:
# Initialize the optimizer
options={'c1': 0.5, 'c2': 0.3, 'w': 0.9}
optimizer = GlobalBestPSO(n_particles= 3, dimensions= 4, options=options, bounds=bounds)

# Run the optimization
best_costs, best_hyperparams = optimizer.optimize(objective_function, iters= 1)

# Print the best hyperparameters found
print("Best position:", best_hyperparams)
print("Best cost:", best_costs)

2024-01-25 12:02:28,387 - pyswarms.single.global_best - INFO - Optimize for 1 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/1Some weights of the model checkpoint at LiYuan/amazon-review-sentiment-analysis were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


learning_rate: 0.0016880306737898222, batch_size: 222, weight_decay: 0.0009419545918603944, num_epochs: 1
epoch No. : 0, Devset Accuracy : 0.8, Devset f1_score : 0.79667, Average loss: 1.32128



Some weights of the model checkpoint at LiYuan/amazon-review-sentiment-analysis were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


learning_rate: 0.009231734454570178, batch_size: 190, weight_decay: 9.018604274760619e-05, num_epochs: 1
epoch No. : 0, Devset Accuracy : 0.1, Devset f1_score : 0.05, Average loss: 1.67766



Some weights of the model checkpoint at LiYuan/amazon-review-sentiment-analysis were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
pyswarms.single.global_best: 100%|██████████|1/1, best_cost=2.58
2024-01-25 12:08:17,363 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 2.5765079823497947, best pos: [1.68803067e-03 2.22008745e+02 9.41954592e-04 1.20808781e+00]


learning_rate: 0.008593180470999665, batch_size: 28, weight_decay: 0.00046871301764920376, num_epochs: 1
epoch No. : 0, Devset Accuracy : 0.1, Devset f1_score : 0.06667, Average loss: 1.4169

Best position: [1.68803067e-03 2.22008745e+02 9.41954592e-04 1.20808781e+00]
Best cost: 2.5765079823497947


In [9]:
learning_rate= best_hyperparams[0]
batch_size= int(best_hyperparams[1])
weight_decay= best_hyperparams[2]
num_epochs= int(best_hyperparams[3])

learning_rate, batch_size, weight_decay, num_epochs

(0.0016880306737898222, 222, 0.0009419545918603944, 1)

In [10]:
# Define hyperparameters
dropout_rate = 0.1   # Dropout ratio of the additional hidden layer
hidden_dim = 256   # 256 classes as an additional hidden layer
output_dim = 5    # 5 classes for sentiment analysis

In [11]:

train_dataloader = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)
dev_dataloader = DataLoader(dev_dataset, batch_size= batch_size, shuffle= False)
test_dataloader = DataLoader(test_dataset, batch_size= batch_size, shuffle= False)

# LiYuan Model
bert_model = AutoModel.from_pretrained(checkpoint)

# Freeze the BERT model parameters
for param in bert_model.parameters():
    param.requires_grad = False

# Create an instance of the SentimentClassifier
model = SentimentClassifier(bert_model, hidden_dim, output_dim, dropout_rate)

# Define the optimizer for training the softmax layer
optimizer = optim.Adam(model.parameters(), lr= learning_rate, weight_decay= weight_decay)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Train the model
best_dev_accuracy = 0.0
best_model_state_dict = None
Validation_results= []

for epoch in range(num_epochs):
    # Training loop
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, batch_labels = batch
        
        input_ids= input_ids.to(device)
        attention_mask= attention_mask.to(device)
        batch_labels= batch_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids= input_ids, attention_mask= attention_mask)
        loss = criterion(outputs, batch_labels)  
        loss.backward()
        optimizer.step()
        
        
    # Validation loop
    model.eval()
    dev_correct = 0
    total_dev = 0
    y_true = []
    y_pred = []
    loss_epoch= []
    with torch.no_grad():
        for batch in dev_dataloader:
            input_ids, attention_mask, batch_labels = batch
            
            input_ids= input_ids.to(device)
            attention_mask= attention_mask.to(device)
            batch_labels= batch_labels.to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs
            _, predicted = torch.max(logits, 1)

            # Append true labels and predicted labels for later use
            y_true.extend(batch_labels.tolist())
            y_pred.extend(predicted.tolist())
            
            # Calculate the loss
            loss = criterion(logits, batch_labels)
            loss_epoch.append(loss)        
        
    # Calculate accuracy and F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    
    # Calculate the average loss
    loss_epoch_np = [tensor.cpu().detach().numpy() for tensor in loss_epoch]
    average_loss= np.mean(loss_epoch_np)
    
    print(f'epoch No. : {epoch}, Devset Accuracy : {round(accuracy,5)}, Devset f1_score : {round(f1,5)}, Average loss: {round(average_loss.tolist(),5)}')
    
    Validation_results.append([accuracy, f1, average_loss])
    
    if accuracy > best_dev_accuracy:
        best_dev_accuracy = accuracy
        # Save the best model (optional)
        best_model_state_dict = model.state_dict()   

Some weights of the model checkpoint at LiYuan/amazon-review-sentiment-analysis were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


epoch No. : 0, Devset Accuracy : 0.8, Devset f1_score : 0.71111, Average loss: 1.38346


In [12]:
# Saving Testset Results
data = {
    'Validation_results': Validation_results,
}
df = pd.DataFrame(data)
df.to_csv('Outputs/Main-9-LiYuan-Two layers-OPT_Validation_results.csv', index= False)

In [13]:
# Load the best model state dict
if best_model_state_dict is not None:
    model.load_state_dict(best_model_state_dict)
    
    # Define the directory path to save the model
    save_path = 'Saved Models/Main-9-LiYuan-Two layers-OPT.pth'  

    # Save the model state dictionary and other relevant information
    torch.save({
        'model_state_dict': best_model_state_dict,
        'tokenizer': tokenizer  
    }, save_path)

In [14]:
# Evaluate on the test set
model.eval()
y_true_test = []
y_pred_test = []
loss_epoch= []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, batch_labels = batch
        
        input_ids= input_ids.to(device)
        attention_mask= attention_mask.to(device)
        batch_labels= batch_labels.to(device)
        
        outputs = model(input_ids= input_ids, attention_mask= attention_mask)
        logits = outputs
        _, predicted = torch.max(logits, 1)

        # Append true labels and predicted labels for later use
        y_true_test.extend(batch_labels.tolist())
        y_pred_test.extend(predicted.tolist())
        
        # Calculate the loss
        loss = criterion(logits, batch_labels)
        loss_epoch.append(loss)

# Calculate accuracy and F1 score for the test set
test_accuracy = accuracy_score(y_true_test, y_pred_test)
test_f1 = f1_score(y_true_test, y_pred_test, average='weighted')

# Calculate the average loss
loss_epoch_np = [tensor.cpu().detach().numpy() for tensor in loss_epoch]
average_loss= np.mean(loss_epoch_np)

print(f"Testset accuracy: {round(test_accuracy,5)} , Testset F1 score: {round(test_f1,5)}, Average loss: {round(average_loss.tolist(),5)}")
Test_results= [test_accuracy, test_f1, average_loss]

Testset accuracy: 0.6 , Testset F1 score: 0.45, Average loss: 1.43218


In [15]:
# Saving Testset Results
data = {
    'Test_results': Test_results
}
df = pd.DataFrame(data)
df.to_csv('Outputs/Main-9-LiYuan-Two layers-OPT_Test_results.csv', index= False)